In [1]:
from ome_zarr.io import parse_url
import zarr

from os.path import join

import mobie
from mobie.metadata.project_metadata import create_project_metadata, add_dataset
from mobie.metadata.dataset_metadata import create_dataset_structure, create_dataset_metadata

from faim_hcs.mobie import add_wells_to_project

from os.path import exists
import shutil

from faim_hcs.mobie import add_wells_to_project

# MoBIE Project

MoBIE is a Fiji viewer to visualize large data like ome-zarr HCS plates. A MoBIE project can hold multiple datasets and each dataset consists of views of different image sources, which can be linked to tabular data. For more information check out the [MoBIE Tutorials]().

Citation: [Pape, Constantin, et al. "MoBIE: a Fiji plugin for sharing and exploration of multi-modal cloud-hosted big image data." Nature Methods (2023): 1-2.](https://www.nature.com/articles/s41592-023-01776-4)

In [2]:
# The MoBIE project folder and name of the plate/dataset
mobie_project_folder = "./mobie-example-project"

In [3]:
if exists(mobie_project_folder):
    # Remove if it already exists.
    shutil.rmtree(mobie_project_folder)

# Create a new empty MoBIE project
create_project_metadata(mobie_project_folder)

## Add 3D OME-Zarr

In [4]:
dataset_name = "Projection-Mix"

In [5]:
# Create a new dataset
create_dataset_structure(mobie_project_folder, dataset_name, file_formats=["ome.zarr"])

'./mobie-example-project/Projection-Mix'

In [6]:
# Fill in dataset metadata
create_dataset_metadata(
    dataset_folder=join(mobie_project_folder, dataset_name), 
    description="A 3D projection mix example dataset.",
    is2d=False,
)

In [7]:
# Update MoBIE project by adding the new dataset.
add_dataset(mobie_project_folder, dataset_name, is_default=True)

In [8]:
# OME-Zarr plate we want to add to a MoBIE project
plate_path = "./zarr-files/Projection-Mix.zarr/"
store = parse_url(plate_path, mode="r").store
    
plate = zarr.group(store=store)

In [9]:
add_wells_to_project(
    plate=plate,
    dataset_folder=join(mobie_project_folder, dataset_name),
    well_group="0",
    view_name="default",
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
add_wells_to_project(
    plate=plate,
    dataset_folder=join(mobie_project_folder, dataset_name),
    well_group="0/projections",
    view_name="Projections",
    label_suffix="_projection",
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Validate MoBIE project integrety
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Projection-Mix: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 67.79it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]

The project at ./mobie-example-project is a valid MoBIE project.


## Add 2D OME-Zarr

In [12]:
dataset_name = "Single-Plane"

In [13]:
# Create a new dataset
create_dataset_structure(mobie_project_folder, dataset_name, file_formats=["ome.zarr"])

'./mobie-example-project/Single-Plane'

In [14]:
# Fill in dataset metadata
create_dataset_metadata(
    dataset_folder=join(mobie_project_folder, dataset_name), 
    description="A 2D dataset.",
    is2d=True,
)

In [15]:
# Update MoBIE project by adding the new dataset.
add_dataset(mobie_project_folder, dataset_name, is_default=True)

In [16]:
# OME-Zarr plate we want to add to a MoBIE project
plate_path = "zarr-files/Single-Plane.zarr/"
store = parse_url(plate_path, mode="r").store
    
plate = zarr.group(store=store)

In [17]:
add_wells_to_project(
    plate=plate,
    dataset_folder=join(mobie_project_folder, dataset_name),
    well_group="0",
    view_name="default",
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
# Validate MoBIE project integrety
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Projection-Mix: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 72.77it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]
Check views for dataset Single-Plane: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 36.53it/s]
Check view files for dataset Single-Plane: 0it [00:00, ?it/s]

The project at ./mobie-example-project is a valid MoBIE project.
